## CmdStanPyの動作確認

### CmdStanPy公式の"Hello World"

In [1]:
# import packages
import os
from cmdstanpy import cmdstan_path, CmdStanModel

# specify Stan program file
bernoulli_stan = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.stan')

# instantiate the model; compiles the Stan program as needed.
bernoulli_model = CmdStanModel(stan_file=bernoulli_stan)

# inspect model object
print(bernoulli_model)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CmdStanModel: name=bernoulli
	 stan_file=/root/.cmdstan/cmdstan-2.30.0/examples/bernoulli/bernoulli.stan
	 exe_file=/root/.cmdstan/cmdstan-2.30.0/examples/bernoulli/bernoulli
	 compiler_options=stanc_options={}, cpp_options={}


In [2]:
# specify data file
bernoulli_data = os.path.join(cmdstan_path(), 'examples', 'bernoulli', 'bernoulli.data.json')

# fit the model
bern_fit = bernoulli_model.sample(data=bernoulli_data, output_dir='.')

# printing the object reports sampler commands, output files
print(bern_fit)

18:10:36 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status
chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status


chain 4 |          | 00:00 Status

chain 1 |█████▉    | 00:00 Iteration: 1001 / 2000 [ 50%]  (Sampling)
chain 2 |█████▉    | 00:00 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 4 |▍         | 00:00 Status


chain 4 |██████▊   | 00:00 Iteration: 1200 / 2000 [ 60%]  (Sampling)


chain 1 |██████████| 00:01 Sampling completed                       
chain 2 |██████████| 00:01 Sampling completed                       
chain 3 |██████████| 00:01 Sampling completed                       
chain 4 |██████████| 00:01 Sampling completed                       


18:10:37 - cmdstanpy - INFO - CmdStan done processing.



CmdStanMCMC: model=bernoulli chains=4['method=sample', 'algorithm=hmc', 'adapt', 'engaged=1']
 csv_files:
	/workdir/demo/bernoulli-20220709181036_1.csv
	/workdir/demo/bernoulli-20220709181036_2.csv
	/workdir/demo/bernoulli-20220709181036_3.csv
	/workdir/demo/bernoulli-20220709181036_4.csv
 output_files:
	/workdir/demo/bernoulli-20220709181036_0-stdout.txt
	/workdir/demo/bernoulli-20220709181036_1-stdout.txt
	/workdir/demo/bernoulli-20220709181036_2-stdout.txt
	/workdir/demo/bernoulli-20220709181036_3-stdout.txt


In [3]:
bern_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-7.281130,0.021597,0.771942,-8.799580,-6.983700,-6.750230,1277.52,616.861,1.004490
theta,0.251705,0.003143,0.120175,0.079998,0.239871,0.472756,1462.05,705.961,0.999806


In [4]:
bern_fit.diagnose()

'Processing csv files: /workdir/demo/bernoulli-20220709181036_1.csv, /workdir/demo/bernoulli-20220709181036_2.csv, /workdir/demo/bernoulli-20220709181036_3.csv, /workdir/demo/bernoulli-20220709181036_4.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.\n'

### 8schools
- pystanとの速度比較

In [5]:
%%time
model = CmdStanModel(stan_file="8schools.stan")

18:11:46 - cmdstanpy - INFO - compiling stan file /workdir/demo/8schools.stan to exe file /workdir/demo/8schools
18:12:01 - cmdstanpy - INFO - compiled model executable: /workdir/demo/8schools


CPU times: user 3.08 ms, sys: 3.32 ms, total: 6.4 ms
Wall time: 14.4 s


- 15s程度でコンパイルできた。
- pystan2では1min超を要したから、確かにコンパイルはかなり高速と言える。
  - コンパイルしたモデルは保存されており、2回目以降は数秒で読み込める。(`8schools`と`8schools.hpp`を削除すれば初回同様となる)

In [6]:
%%time
data = "8schools.data.json"
fit = model.sample(data=data, iter_sampling=1000, iter_warmup=500)

18:12:13 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status
chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status


chain 4 |          | 00:00 Status

chain 3 |██████████| 00:00 Iteration: 1400 / 1500 [ 93%]  (Sampling)


chain 1 |██████████| 00:00 Sampling completed1500 [  6%]  (Warmup)

chain 2 |██████████| 00:00 Sampling completed
chain 3 |██████████| 00:00 Sampling completed                       
chain 4 |██████████| 00:00 Sampling completed                     


18:12:13 - cmdstanpy - INFO - CmdStan done processing.
18:12:13 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 2 divergent transitions (0.2%)
	Chain 4 had 1 divergent transitions (0.1%)
	Use function "diagnose()" to see further information.



CPU times: user 79 ms, sys: 22.8 ms, total: 102 ms
Wall time: 269 ms


- 200ms強を要しており、微差ではあるがPyStan2（163ms）よりやや遅い？

In [7]:
fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-4.852800,0.076550,2.595570,-9.493650,-4.604730,-1.05976,1149.68,6684.21,1.004310
mu,7.804230,0.112907,4.987970,-0.211724,7.779770,15.77010,1951.66,11346.90,1.002200
tau,6.560580,0.145619,5.465320,0.502171,5.227680,17.28180,1408.62,8189.66,1.001100
eta[1],0.402509,0.015871,0.973413,-1.234160,0.423385,1.99472,3761.51,21869.30,0.999815
eta[2],0.020204,0.014403,0.867166,-1.409680,0.042121,1.41954,3624.87,21074.80,1.000120
eta[3],-0.210209,0.014796,0.957735,-1.798150,-0.218614,1.35151,4189.77,24359.10,1.000170
eta[4],0.006349,0.013795,0.886149,-1.439310,0.009969,1.46448,4126.61,23991.90,1.000400
eta[5],-0.334944,0.014491,0.864738,-1.701340,-0.343935,1.11143,3560.82,20702.50,0.999918
eta[6],-0.208016,0.013925,0.884984,-1.691420,-0.216465,1.24464,4039.16,23483.50,1.000020


In [9]:
print(fit.diagnose())

Processing csv files: /tmp/tmp2ujed_qy/8schoolsj4xv6kqk/8schools-20220709181213_1.csv, /tmp/tmp2ujed_qy/8schoolsj4xv6kqk/8schools-20220709181213_2.csv, /tmp/tmp2ujed_qy/8schoolsj4xv6kqk/8schools-20220709181213_3.csv, /tmp/tmp2ujed_qy/8schoolsj4xv6kqk/8schools-20220709181213_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
3 of 4000 (0.07%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete.

